<a href="https://colab.research.google.com/github/djmary-k/Py16DS_HW03/blob/main/Hw3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Homework 03, Module 03, Data Science.
Дане домашнє завдання буде повністю пов'язане з лінійною регресією та її реалізацією.

Розіб'ємо наше домашнє завдання на декілька частин:
* напишіть функцію гіпотези лінійної регресії у векторному вигляді;
* створіть функцію для обчислення функції втрат у векторному вигляді;
* реалізуйте один крок градієнтного спуску;

In [22]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

In [23]:
def cost(h, y):
  return np.mean((h-y)**2)/2

class CustomLinearRegression:
  def __init__(self, lr: float = 0.01, thr=0.000000001, n_epochs = 1000):
    # lr - learning rate(коефіцієнт навчання, який відповідає за те, наскільки великий крок буде робити алгоритм на кожній ітерації)
    # thr - критерій зупинки навчання або трешхолд
    # n_epoch - кількість епох\ кроки градієнтного спуску (кількість ітерацій \ num_iter)
    self.lr = lr # alfa
    self.thr = thr # epsilon
    self.n_epochs = n_epochs # num_iter
    # self.w = np.array([np.random.uniform(-5,5)])
    self.w = np.array([np.random.uniform(-5,5), np.random.uniform(-5,5), np.random.uniform(-5,5)])

  def predict(self, X): # гіпотетична модель h(x)
    return self.w @ X.T

  def update_weights(self, X, y): #реалізація одиного кроку градієнтного спуску grad_step
    m = len(y)
    h = self.predict(X)
    self.w = self.w - self.lr / m * X.T @ (h-y)

  def fit(self, X, y): # цикл із кроків градієнтного спуску grade_descent
    last_cost = 10000000 #рандомне величезне число
    for i in range(self.n_epochs):
      self.update_weights(X, y)
      new_cost = cost(self.predict(X), y)

      if i % (self.n_epochs//1000) == 0:
        print(f'cost function = {last_cost}')

      if abs(last_cost - new_cost) < self.thr: # зупинка навчання
        break

      last_cost = new_cost



* знайдіть найкращі параметри *w* для датасету прогнозуючу ціну на будинок залежно від площі, кількості ванних кімнат та кількості спалень

In [24]:
data_housing = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Housing.csv')
data_housing.head()

,price,area,bedrooms,bathrooms,stories,mainroad,guestroom,basement,hotwaterheating,airconditioning,parking,prefarea,furnishingstatus
0,13300000,7420,4,2,3,yes,no,no,no,yes,2,yes,furnished
1,12250000,8960,4,4,4,yes,no,no,no,yes,3,no,furnished
2,12250000,9960,3,2,2,yes,no,yes,no,no,2,yes,semi-furnished
3,12215000,7500,4,2,2,yes,no,yes,no,yes,3,yes,furnished
4,11410000,7420,4,1,2,yes,yes,yes,no,yes,2,no,furnished


In [25]:
# data_housing.plot(x='area', y='price', kind='scatter')

In [27]:
# normalization data: 'X' and 'y'

def normalization(data):
  mean_data = np.mean(data)
  value_range = np.max(data) - np.min(data)
  result = []

  for x in data:
    norm_x = (x - mean_data) / value_range
    result.append(norm_x)

  return result

In [28]:
# # y - Vector [prices]
# # X - Matrix [area]

# norm_y = np.array(normalization(data_housing.price))
# norm_x = pd.DataFrame()
# norm_x['area'] = normalization(data_housing.area)

In [29]:
# y - Vector [prices]
# X - Matrix [area, bathrooms, bedrooms]

norm_y = np.array(normalization(data_housing.price))
norm_x = pd.DataFrame()
norm_x['area'] = normalization(data_housing.area)
norm_x['bathrooms'] = normalization(data_housing.bathrooms)
norm_x['bedrooms'] = normalization(data_housing.bedrooms)

In [100]:
# cust_regression = CustomLinearRegression(lr=0.1, thr=0.00000001)
cust_regression = CustomLinearRegression(lr=0.1, thr=0.000001)
cust_regression.fit(norm_x, norm_y)

cost function = 10000000
cost function = 0.050685230685267736
cost function = 0.05036445781623716
cost function = 0.050046038859246154
cost function = 0.04972995649272921
cost function = 0.049416193522751554
cost function = 0.0491047328820682
cost function = 0.048795557629189835
cost function = 0.04848865094745563
cost function = 0.048183996144112895
cost function = 0.0478815766494035
cost function = 0.04758137601565703
cost function = 0.04728337791639073
cost function = 0.04698756614541589
cost function = 0.04669392461595101
cost function = 0.046402437359741436
cost function = 0.04611308852618544
cost function = 0.0458258623814668
cost function = 0.045540743307693775
cost function = 0.04525771580204444
cost function = 0.04497676447591827
cost function = 0.044697874054094054
cost function = 0.04442102937389392
cost function = 0.04414621538435366
cost function = 0.0438734171453991
cost function = 0.04360261982702847
cost function = 0.043333808708500926
cost function = 0.0430669691775310

In [101]:
cust_regression.w

area         0.548741
bathrooms    0.363162
bedrooms     0.264020
dtype: float64

* знайдіть ці ж параметри за допомогою аналітичного рішення

In [121]:
from sklearn.linear_model import LinearRegression
regressor = LinearRegression().fit(norm_x, norm_y)
# коефіцієнти
print(regressor.coef_)
# зсув
print(regressor.intercept_)

[0.47714269 0.36001286 0.17611257]
7.014204135081996e-17


* порівняйте отримані результати

Отже, параметри *w* для даного датасету:
* отримані за допомогою кастомної реалізації

**area         0.548741**
**bathrooms    0.363162**
**bedrooms     0.264020**

* отримані за допомогою бібліотеки scikit learn:

**[0.47714269 0.36001286 0.17611257]**


*Щоб визначити який результат кращий, спрогнозуємо ціни за допомогою отриманих параметрів w і порівняємо їх з нашим датасетом.*

In [120]:
# Split the data into training/testing sets
X_train = data_housing.loc[:500, ['area', 'bathrooms', 'bedrooms']]
X_test = data_housing.loc[501:, ['area', 'bathrooms', 'bedrooms']]

# Split the targets into training/testing sets
y_train = data_housing.loc[:500, 'price']
y_test = data_housing.loc[501:, 'price']



In [ ]:
# # Create linear regression object
# regr = LinearRegression()

# # Train the model using the training sets
# regr.fit(X_train, y_train)

In [ ]:
# # The coefficients
# print("Coefficients: \n", regr.coef_)

In [122]:
# Make predictions using the testing set
y_pred = regressor.predict(X_test)
y_pred

array([1160.34508625, 1661.16879774, 1909.45910889, 1520.41170431,
       1909.45910889, 1389.19746468, 1718.42592049, 2100.14007215,
       1718.78593334, 1375.05929656, 1518.20210343, 1432.14030674,
       2100.31618472, 1432.31641931, 1532.51638412, 1546.65455224,
       1432.14030674, 1670.71165153, 2310.08285557, 3674.71094777,
       1735.12591462, 1181.81650728, 1331.22115185, 1558.10597679,
       1737.51162807, 1518.02599086,  876.74621607, 1894.79260306,
       1895.1448282 ,  931.31659525, 2529.74460534, 1432.14030674,
       1146.03080556, 1432.49253188, 1603.91167499, 1633.06857407,
        812.03092285, 1741.80591228, 1427.36887984, 1432.14030674,
       1146.03080556, 1727.96877428, 1389.37357724, 1837.88770545])

In [123]:
data_housing.price.tail()

540    1820000
541    1767150
542    1750000
543    1750000
544    1750000
Name: price, dtype: int64

In [114]:
cust_regression.predict(X_test)

501    1334.595026
502    1910.508700
503    2196.117863
504    1748.630205
505    2196.117863
506    1597.726523
507    1976.357579
508    2415.350107
509    1976.720741
510    1581.528323
511    1746.150522
512    1647.113183
513    2415.614127
514    1647.377203
515    1762.612741
516    1778.810941
517    1647.113183
518    1921.483513
519    2656.795998
520    4226.194287
521    1995.563502
522    1359.288356
523    1531.122624
524    1791.980717
525    1998.307206
526    1745.886502
527    1008.379054
528    2179.127604
529    2179.655643
530    1071.199509
531    2909.480722
532    1647.113183
533    1318.132806
534    1647.641222
535    1844.659821
536    1878.376320
537     934.014344
538    2003.245871
539    1641.625776
540    1647.113183
541    1318.132806
542    1987.332392
543    1597.990543
544    2113.806764
dtype: float64

# Питання до метора

1. Не зовсім зрозуміла чому ми задаєм **self.w** ось таким чином. Поясніть, будь ласка.

In [ ]:
class CustomLinearRegression:
  def __init__(self, lr: float = 0.01, thr=0.000000001, n_epochs = 1000):
   self.w = np.array([np.random.uniform(-5,5), np.random.uniform(-5,5), np.random.uniform(-5,5)])

2. Чому задаєм початкові ваги як рандомне величезне число? І на що орієнтуватись при виборі даного числа? (тобто, як його вибрати?)

In [ ]:
def fit(self, X, y):
    last_cost = 10000000 #рандомне величезне число

3. Це питання стосуєтся прогнозування цін. Чому я отримала дивні результати, які дуже відрізняються від цін датасету. В чому помилка, що не так зроблено?